In [21]:
# pip install openpyxl
import pandas as pd
import numpy as np
import os
import openpyxl

In [22]:
print(os.getcwd())

/workspaces/Product-inventory-analytics


## Проведем аналогичные преобразования

In [23]:
Oststki = pd.read_excel('Остатки.xlsx', header=10)

# Заменили пустые значения нулями
Oststki = Oststki.fillna(0)

Oststki.drop(columns = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Комментарии', 'остаток > расхода', 'стоимость за 1 ед'], axis=1, inplace = True)

Oststki.rename(columns = {'Номенклатура, № по кат., Производитель':'Name', 
                          'Количество (в ед. хранения)':'Нач_ост_кол',
                          'Сумма (в рег. валюте)':'Нач_ост_сумм',
                          'Количество (в ед. хранения).1':'Приход_Кол',
                          'Сумма (в рег. валюте).1':'Приход_Сумм',
                          'Количество (в ед. хранения).2':'Расход_кол',
                          'Сумма (в рег. валюте).2':'Расход_сумм',
                          'Количество (в ед. хранения).3':'Кон_ост_кол',
                          'Сумма (в рег. валюте).3':'Кон_ост_сумм',
                          }, inplace = True )

In [24]:
# Отберем данные для отдельной таблице по Угданскому Чатц
start = Oststki.index [Oststki['Name']== "Угданский ЧАТЦ  склад " ]. tolist ()[0]
end = Oststki.index [Oststki['Name']== "Основной ЧАТЦ" ]. tolist ()[0]

In [25]:
# Функция для отбора только данным по Угданский Чатц
list = [i for i in range(start, end+1)]
Ugdan = Oststki.take(list, axis = 0)

## Анализ групп

In [26]:
# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["ЧАТЦ"]
Ugdan = Ugdan[~Ugdan.Name.str.contains('|'.join(rule))]
Ugdan = Ugdan[~Ugdan.Name.str.contains('|'.join(rule_2))]

In [27]:
Ugdan.tail()

,Name,Нач_ост_кол,Нач_ост_сумм,Приход_Кол,Приход_Сумм,Расход_кол,Расход_сумм,Кон_ост_кол,Кон_ост_сумм
7963,"Шпилька колесная BPW M22*1.5*89/54 070246 , 07...",10.0,2155.18,1.0,300.00,2.0,640.00,9.0,1815.18
7964,"Шприц маслозаливной (пластиковый) 13430, 13430,",0.0,0.00,6.0,552.00,6.0,552.00,0.0,0.00
7965,Шприц рычажно-плунжерный 500 мл БелАК БАК00019...,1.0,596.11,19.0,12793.11,18.0,11995.91,2.0,1393.31
7966,"Шприц-масленка пластмассовая SH0750, SH0750,",4.0,440.00,0.0,0.00,4.0,440.00,0.0,0.00
7967,"Эпоксидный клей усиленный AVS AVK-129 17922, 1...",0.0,0.00,12.0,1474.00,10.0,1228.00,2.0,246.00


### Информация по группам

In [28]:
# Группа 1

Count_group_1 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0')['Name'].count()
Sum_group_1 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0')['Кон_ост_сумм'].sum()

Count_group_1_5 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')['Name'].count()
Sum_group_1_5 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')['Кон_ост_сумм'].sum()

Count_group_1_10 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')['Name'].count()
Sum_group_1_10 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')['Кон_ост_сумм'].sum()

In [29]:
# Группа 2
Count_group_2 = Ugdan.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')['Name'].count()
Sum_group_2 = Ugdan.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')['Кон_ост_сумм'].sum()

# Группа 3
Count_group_3 = Ugdan.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_кол > Расход_кол')['Name'].count()
Sum_group_3 = Ugdan.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_кол > Расход_кол')['Кон_ост_сумм'].sum()

# Группа 4
Count_group_4 = Ugdan.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')['Name'].count()
Sum_group_4 = Ugdan.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')['Кон_ост_сумм'].sum()

# Группа 5
Count_group_5 = Ugdan.query('Расход_кол >=3 and Кон_ост_кол == 0')['Name'].count()
Sum_group_5 = Ugdan.query('Расход_кол >=3 and Кон_ост_кол == 0')['Расход_сумм'].sum()

In [30]:
groups = pd.DataFrame({'Наименование': ['группа 1- "Мертвый" склад', 'группа 1 - свыше 5 тыс', 'группа 1 - свыше 10 тыс', 'группа 2 - Бестолковые закупки', 'группа 3 - Излишние запасы', 'группа 4 - Проверка спроса', 'группа 5 - Нет ходового товара'], 
                    'Сколько позиций.': [Count_group_1, Count_group_1_5, Count_group_1_10, Count_group_2, Count_group_3, Count_group_4, Count_group_5], 
                    'Сумма.': [Sum_group_1, Sum_group_1_5, Sum_group_1_10, Sum_group_2, Sum_group_3, Sum_group_4, Sum_group_5]})
groups = groups.set_index('Наименование').round(2)

groups

,Сколько позиций.,Сумма.
Наименование,,
"группа 1- ""Мертвый"" склад",455,2124984.51
группа 1 - свыше 5 тыс,92,981347.38
группа 1 - свыше 10 тыс,29,416146.40
группа 2 - Бестолковые закупки,2,8258.18
группа 3 - Излишние запасы,162,556734.29
группа 4 - Проверка спроса,122,634063.73
группа 5 - Нет ходового товара,221,1408587.22


## Создадим отдельные таблицы для более датального просмотра товарных позиций

In [31]:
Group_1 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0')
Group_1_5 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')
Group_1_10 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')

Group_2 = Ugdan.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')
Group_3 = Ugdan.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_сумм > Расход_сумм')
Group_4 = Ugdan.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')
Group_5 = Ugdan.query('Расход_кол >=3 and Кон_ост_кол == 0')

# Выгрузка итоговой таблицы

In [32]:
# Отредактируем файл Metrics для добавления обновленных листов
wb = openpyxl.load_workbook('Metrics.xlsx') 

In [33]:
sheet = wb.sheetnames
print(sheet)

['Sheet1', 'Группа 1', 'Группа 1 (>5)', 'Группа 1 (>10)', 'Группа 2', 'Группа 3', 'Группа 4', 'Группа 5']


In [34]:
with pd.ExcelWriter('Metrics.xlsx', mode='a', if_sheet_exists= 'replace') as writer:  
    Group_1.to_excel(writer, sheet_name='Группа 1',index = False)
    Group_1_5.to_excel(writer, sheet_name='Группа 1 (>5)',index = False)
    Group_1_10.to_excel(writer, sheet_name='Группа 1 (>10)',index = False)
    Group_2.to_excel(writer, sheet_name='Группа 2',index = False)
    Group_3.to_excel(writer, sheet_name='Группа 3',index = False)
    Group_4.to_excel(writer, sheet_name='Группа 4',index = False)
    Group_5.to_excel(writer, sheet_name='Группа 5',index = False)